# requests 포스트 요청
- [위시캣](https://www.wishket.com)
- IT 아웃소싱 사이트
    - 프리랜서 - 개발사를 연결해 주는 사이트

1. https://www.wishket.com/accounts/login/ (로그인 사이트) 에서 크롬개발자 도구를 키고 로그인 하면서 Network 흐름을 본다.
![img](post_img/위시캣_01.jpg)
![img](post_img/위시캣_02.jpg)
2. 그냥 연결하면 403 뜬다. - csrf 토큰 쿠키가 필요하고 헤더 정보를 설정 해야 한다.
3. csrf 는 보안을 위해 서버가 전송하는 쿠키값이며 요청할 때 마다 요청파라미터로 전송된다. 그런데 우린 웹브라우저가 아니므로 저장이 안된다. 그리고 이건 실행할 때 마다 바뀐다.
    - 일단 get방식으로 연결한다. 그럼 csrf 쿠키를 respone로 부터 읽을 수 있다.
4. header 정보가 어쩌구 저쩌구 에러난다. (에러메세지)
    - 확인해보면 user-agent가 python-requests라고 가므로 에러가 난다.
    - 구글에서 my user agent로 조호하거나 개발자 도구에서 확인
5. referer 필요 (header에 referer 조회)    

In [48]:
import requests
from bs4 import BeautifulSoup
import json
url = 'https://www.wishket.com/accounts/login/'
user_info = json.load(open('wishket_pwd.txt')) #id/pwd

In [55]:
with requests.Session() as s:
    # csrf 쿠키를 읽기 위해서 일단 연결해 본다.
    s.get(url)
    csrf_cookie = s.cookies['csrftoken'] #쿠키이름은 개발자도구 application 탭에서 확인
    print(s.cookies['csrftoken']) #실행할 때마다 바뀜
    PARAM_INFO = {
        'identification':user_info['id'],
        'password':user_info['pwd'],
        'csrfmiddlewaretoken':csrf_cookie        
    }
#     print(s.headers)
    HEADER_INFO = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
        'Referer': 'https://www.wishket.com/accounts/login/'
    }
    res = s.post(url, data = PARAM_INFO, headers=HEADER_INFO )
    print(res.status_code)
    # 403 권한 없는 경우 발생
    
    # 위시켓 히스토리 가져오기
    if res.status_code == 200:
        soup = BeautifulSoup(res.text)
       
        tr_list = soup.select('table.table.table-responsive > tbody > tr')
        print(tr_list)
        for tr in tr_list:
            print(tr.find('th').text.strip())
            print(tr.find('td').text.strip())

HG0cAQsZ9uwjVD6gyImcn8LOLa3TBPmy
{'identification': 'kgmyh92', 'password': 'mptres9273', 'csrfmiddlewaretoken': 'HG0cAQsZ9uwjVD6gyImcn8LOLa3TBPmy'}
200
[<tr><th>지원한 프로젝트</th><td>0 <span class="append-unit">건</span></td></tr>, <tr><th>계약한 프로젝트</th><td>0 <span class="append-unit">건</span></td></tr>, <tr><th>완료한 프로젝트</th><td>0 <span class="append-unit">건</span></td></tr>]
지원한 프로젝트
0 건
계약한 프로젝트
0 건
완료한 프로젝트
0 건
